In [8]:
from pmaw import PushshiftAPI
import pandas as pd

import datetime
import time

In [2]:
api = PushshiftAPI()

In [13]:
comments = api.search_comments(subreddit="uci", limit=1000)
comment_list = [comment for comment in comments]
comment_list[0]

{'all_awardings': [],
 'archived': False,
 'associated_award': None,
 'author': 'PantasticNerd',
 'author_created_utc': 1562731378,
 'author_flair_background_color': None,
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_template_id': None,
 'author_flair_text': None,
 'author_flair_text_color': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_44d4gky7',
 'author_patreon_flair': False,
 'author_premium': False,
 'body': 'Congrats!',
 'can_gild': True,
 'collapsed': False,
 'collapsed_because_crowd_control': None,
 'collapsed_reason': None,
 'collapsed_reason_code': None,
 'comment_type': None,
 'controversiality': 0,
 'created_utc': 1649508002,
 'distinguished': None,
 'edited': False,
 'gilded': 0,
 'gildings': {},
 'id': 'i4108jx',
 'is_submitter': False,
 'link_id': 't3_tzkjne',
 'locked': False,
 'no_follow': True,
 'parent_id': None,
 'permalink': '/r/UCI/comments/tzkjne/tag_decision_got_accepted_d/i4108jx/',
 'score': 1,
 'score_hidden': Fa

In [31]:
df = pd.DataFrame(comment_list)
df.to_csv('redditsample1000.csv', index=False)

# cleaned data frame
keys = ["id", "link_id", "subreddit_id", "parent_id", "subreddit", "body", "score", "created_utc", "utc_datetime_str"]  
df_1 = df[keys].copy()
df_1.to_csv('redditsample1000cleaned.csv', index=False)

In [35]:
## GOAL: FIGURE OUT TIME
comments1 = api.search_comments(subreddit="uci", before = 1617260400, after = 1614585600, limit=1000)
comment_list1 = [comment1 for comment1 in comments1]

# posts = api.search_submissions(subreddit="uci", limit=1000, before=1675238400000)#, after=1672560000000)
# posts_list = [post for post in posts]

# posts_list

In [9]:
def get_date_timestamps(start, end, n):
    """
    Generates all unix timestamps for each month between start and end.

    start: datetime.datetime for first day of the first month not being checked
    end: datetime.datetime for first day of the last month to be checked
    """
    periods = []
    curr = start
    delta = datetime.timedelta(days=n)
    
    while curr > end:
        before = time.mktime(curr.timetuple()) # midnight on first day of curr month

        curr -= delta

        after = time.mktime(curr.timetuple()) # midnight on first day of prev month

        periods.append((int(before), int(after)))


    return periods

In [26]:
def log_overflow(start, end, comment_list, filepath):
    if len(comment_list) == 1000:
        with open(filepath, 'a+') as log:
            log.write(str(start))
            log.write(' ')
            log.write(str(end))

In [14]:
## test get_date_timestamps
dates = get_date_timestamps(start=datetime.datetime(2023,3,3), end=datetime.datetime(2023,2,15), n=2)

[(1677830400, 1677657600),
 (1677657600, 1677484800),
 (1677484800, 1677312000),
 (1677312000, 1677139200),
 (1677139200, 1676966400),
 (1676966400, 1676793600),
 (1676793600, 1676620800),
 (1676620800, 1676448000)]

In [15]:
def get_comments(start, end, n, logpath):
    dates = get_date_timestamps(start, end , n)
    start = dates[0][0]
    after = dates[0][1]
    
    comments = api.search_comments(subreddit="uci", before = start, after = after, limit=1000)
    comment_list = [comment for comment in comments]
    log_overflow(start, end, comment_list, logpath)
    comment_df = pd.DataFrame(comment_list)

    for date in dates[1:]:
        start = date[0]
        after = date[1]

        comments = api.search_comments(subreddit="uci", before = start, after = after, limit=1000)
        comment_list = [comment for comment in comments] 
        log_overflow(start, end, comment_list, logpath)
        comment_df = pd.concat([comment_df, pd.DataFrame(comment_list)])
    
    return comment_df

In [18]:
comment_df = get_comments(start=datetime.datetime(2023,3,1), end=datetime.datetime(2023,2,1), n=2)

In [23]:
comment_df

,subreddit_id,author_is_blocked,comment_type,edited,author_flair_type,total_awards_received,subreddit,author_flair_template_id,id,gilded,...,author_flair_background_color,collapsed_because_crowd_control,distinguished,retrieved_utc,updated_utc,body_sha1,nest_level,utc_datetime_str,edited_on,author_cakeday
0,t5_2r7xu,False,None,False,text,0,UCI,None,jagmpu8,0,...,None,None,None,1677693415,1677693416,3997cc1afca33648d030f52b710af43a877770ff,1.0,2023-03-01 07:57:06,NaN,NaN
1,t5_2r7xu,False,None,False,text,0,UCI,None,jaglyfj,0,...,None,None,None,1677693469,1677693470,6a814ddd5dcbfd2323cd83994470b7083b789769,1.0,2023-03-01 07:46:57,NaN,NaN
2,t5_2r7xu,False,None,False,text,0,UCI,None,jaglha9,0,...,None,None,None,1677693505,1677693506,0539d62ef3fd3715cc54b155a4094007cc02da97,1.0,2023-03-01 07:40:38,NaN,NaN
3,t5_2r7xu,False,None,False,text,0,UCI,None,jagl61w,0,...,None,None,None,1677693529,1677693530,f898f00e72ba6ecb805b7137eddd2ae114faa746,1.0,2023-03-01 07:36:38,NaN,NaN
4,t5_2r7xu,False,None,False,text,0,UCI,None,jagl453,0,...,None,None,None,1677693532,1677693533,51a11a44234797bd670c75b8d9c18685d617878b,NaN,2023-03-01 07:35:57,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,t5_2r7xu,False,None,False,text,0,UCI,None,j70utdl,0,...,None,None,None,1675407322,1675407323,df95324d3731305b54241a45fe81bf197282388b,NaN,2023-02-03 06:55:08,NaN,NaN
516,t5_2r7xu,False,None,False,text,0,UCI,None,j70uor2,0,...,None,None,None,1675407229,1675407229,5ee808b395307991d81f25c0ebd6e13beddeb1e3,NaN,2023-02-03 06:53:36,NaN,NaN
517,t5_2r7xu,False,None,False,text,0,UCI,None,j70ru7b,0,...,None,None,None,1675405255,1675405256,8c2e58b5032282440dec3eb8f4f71e29c50c222c,1.0,2023-02-03 06:20:37,NaN,NaN
518,t5_2r7xu,False,None,False,text,0,UCI,None,j70rs23,0,...,None,None,None,1675405209,1675405210,1286af8b0e5c2ac960c93105bcb4bf31cc143e61,1.0,2023-02-03 06:19:57,NaN,NaN


In [22]:
dates = get_date_timestamps(start=datetime.datetime(2023,3,1), end=datetime.datetime(2023,2,1), n=2)
dates

[(1677657600, 1677484800),
 (1677484800, 1677312000),
 (1677312000, 1677139200),
 (1677139200, 1676966400),
 (1676966400, 1676793600),
 (1676793600, 1676620800),
 (1676620800, 1676448000),
 (1676448000, 1676275200),
 (1676275200, 1676102400),
 (1676102400, 1675929600),
 (1675929600, 1675756800),
 (1675756800, 1675584000),
 (1675584000, 1675411200),
 (1675411200, 1675238400)]

In [33]:
# uncleaned data frame: februrary 2023 - march 2023
comment_df.to_csv('redditcommentsamplefeb2023.csv', index=False)

# cleaned data frame
keys = ["id", "link_id", "subreddit_id", "parent_id", "subreddit", "body", "score", "created_utc", "utc_datetime_str"]  
comment_clean_df = comment_df[keys].copy()
comment_clean_df.to_csv('redditcommentsamplefeb2023cleaned.csv', index=False)

In [27]:
def get_posts(start, end, n, logpath):
    dates = get_date_timestamps(start, end , n)
    start = dates[0][0]
    after = dates[0][1]

    posts = api.search_submissions(subreddit="uci", before = start, after = after, limit=1000)
    post_list = [post for post in posts]
    log_overflow(start, end, post_list, logpath)
    post_df = pd.DataFrame(post_list)


    for date in dates[1:]:
        start = date[0]
        after = date[1]

        posts = api.search_submissions(subreddit="uci", before = start, after = after, limit=1000)
        post_list = [post for post in posts]
        log_overflow(start, end, post_list, logpath)
        post_df = pd.concat([post_df, pd.DataFrame(post_list)])
    
    return post_df

In [31]:
post_df = get_posts(start=datetime.datetime(2023,3,1), end=datetime.datetime(2023,2,1), n=5, logpath='postfeb2023.txt')
post_df

,subreddit,selftext,author_fullname,gilded,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,...,is_gallery,gallery_data,url_overridden_by_dest,post_hint,preview,edited_on,poll_data,author_cakeday,crosspost_parent_list,crosspost_parent
0,UCI,I am looking for a male UCI student who wants ...,t2_flq8g2nh,0,Sublease Plaza Verde 4b2b (6/17 - 9/1) for male,[],r/UCI,False,6,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,UCI,Data collection for my research study on Spart...,t2_vmwffcdy,0,Last Call! - Garmin/Spartan Racer Research Study!,[],r/UCI,False,6,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,UCI,Just turned 21 and am looking for some recomme...,t2_1nh5618v,0,Which bars in OC are college-friendly/fun?,[],r/UCI,False,6,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,UCI,[deleted],NaN,0,Which bars in OC are college-friendly/fun?,[],r/UCI,False,6,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,UCI,is there anyone who’s taken inf 133 and is wil...,t2_d1etwznl,0,any INF/SE/BIM/CGS majors willing to hold a spot?,[],r/UCI,False,6,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,UCI,I’m posting this because a guy right next to m...,t2_7jryruyq,0,Wear a mask!!!!!!,[],r/UCI,False,6,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,UCI,Hi everyone! I am making this post in hopes to...,t2_3whi7rkn5,0,Are you looking for housing?!,[],r/UCI,False,6,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,UCI,,t2_21h77kez,0,UCI aesthetics,[],r/UCI,False,6,None,...,NaN,NaN,https://i.redd.it/4mp0ttmuqpfa1.jpg,image,{'images': [{'source': {'url': 'https://previe...,NaN,NaN,NaN,NaN,NaN
180,UCI,,t2_rsno3ze9,0,Y’all be careful the scammer is here 💀poor kid...,[],r/UCI,False,6,None,...,NaN,NaN,https://i.redd.it/ak37utgpzofa1.jpg,image,{'images': [{'source': {'url': 'https://previe...,NaN,NaN,NaN,NaN,NaN


In [32]:
# uncleaned data frame: februrary 2023 - march 2023
post_df.to_csv('redditpostsamplefeb2023.csv', index=False)

# cleaned data frame
# keys = ["id", "link_id", "subreddit_id", "parent_id", "subreddit", "body", "score", "created_utc", "utc_datetime_str"]  
# post_clean_df = post_df[keys].copy()
# post_clean_df.to_csv('redditpostsamplefeb2023cleaned.csv', index=False)